# XGBoost model

This is an XGBoost model, which uses clinical variables identified through data-driven feature selection to predict seizure outcome (seizure-free versus not seizure-free).

The dataframe has - prior to this - been processed in the following manner:

1. Predictors have been selected
2. Categorical predictors have been one-hot encoded
3. Missing data have been dropped (in the case of continuous variables as well as the categorical variable seizure outcome) or labelled as missing (in the case of categorical variables)

In [ ]:
# Import relevant packages

import pandas as pd
import numpy as np
import random
from scipy.stats import binom
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from sklearn import metrics
from statsmodels.stats.proportion import proportion_confint
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import StratifiedKFold 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
# Read in the dataframe

df_sf = pd.read_csv('/home/maria/Desktop/PhD/Predicting-outcome-clinical-paper/Dataframes/Outputs/df_complete_cohort_grouped.csv')

In [ ]:
# Check the dataframe

df_sf.head()

# Step 1: Implement the model

In [ ]:
# Define X (predictors) and y (outcome)

X = df_sf[df_sf.columns[~df_sf.columns.str.contains('one_year_sf')]]
y = df_sf.one_year_sf

# Convert the columns to float 

X = X.astype(np.float)
y = y.astype(np.float)

# Step 2: Test the model using stratified k fold cross-validation

In [ ]:
# Define stratified k fold

skf = StratifiedKFold(n_splits = 10, random_state=1, shuffle=True)

In [ ]:
# Convert the dataframes to arrays

X = np.array(X)
y = np.array(y)

In [ ]:
# Instantiate model

model_xgboost_own = XGBClassifier(base_score = 0.5, booster = 'gbtree', gamma = 0,
                    max_depth = 1, eval_metric = 'error', subsample = 0.9,
                    objective = 'binary:logistic')

In [ ]:
# Perform train and test using stratified k fold cross-validation

scores=np.zeros(len(X))
y_pred=np.zeros(len(y))
y_pred_fold=[]
lst_accu_stratified = []
cm_holder=[]
auc_store = []
for train_index, test_index in skf.split(X, y):
    # split data in stratified fold
    x_train_fold, x_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    # train on the fold
    model_xgboost_own.fit(x_train_fold, y_train_fold)
    scores_fold = model_xgboost_own.predict_proba(x_test_fold)[:,1]
    auc_fold = metrics.roc_auc_score(y_test_fold, scores_fold)
    auc_store.append(auc_fold)
    scores[test_index] = scores_fold
    y_pred[test_index] = model_xgboost_own.predict(x_test_fold)
    lst_accu_stratified.append(model_xgboost_own.score(x_test_fold, y_test_fold))

## Predicted probabilities

In [ ]:
# Plot predicted probability

plt.figure(0)
plt.hist(scores[y==0], label='Not seizure free', color='b', alpha=0.5)
plt.hist(scores[y==1], label='Seizure free', color='k', alpha=0.5)
plt.legend()

## Confusion matrix

In [ ]:
# Plot confusion matrix

confusion_matrix = pd.crosstab(y, y_pred, rownames=['Actual'], colnames=['Predicted'])

x_axis_labels = ['Not seizure-free', 'Seizure-free'] # labels for x-axis
y_axis_labels = ['Not seizure-free', 'Seizure-free'] # labels for y-axis

# Change font 
annot_kws={'fontsize':12, 
           'alpha':0.6, 
           'verticalalignment':'center'
          }

# Create heatmap
sns.heatmap(confusion_matrix, 
            annot=True,
            annot_kws= annot_kws,
            fmt='g',
            xticklabels = x_axis_labels, 
            yticklabels = y_axis_labels)

In [ ]:
# Extract values from confusion matrix

from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y, y_pred)
TP = confusion_matrix[1, 1]
TN = confusion_matrix[0, 0]
FP = confusion_matrix[0, 1]
FN = confusion_matrix[1, 0]

In [ ]:
# Calculate confidence interval for model accuracy

count=(y_pred==y).sum()
nobs=len(y)
ci_low, ci_high = proportion_confint(count,
                                     nobs, alpha=0.05, method='normal')

## Model accuracy

In [ ]:
# Determine how well the model performs

print('List of accuracies obtained from each fold:', 
      lst_accu_stratified)

print('\nMaximum accuracy that can be obtained from this model:',
      round(max(lst_accu_stratified)*100), '%')

print('Minimum accuracy that can be obtained from this model:',
      round(min(lst_accu_stratified)*100), '%')

print('\nOverall model accuracy:',
      round(np.mean(lst_accu_stratified)*100), '%')

print('CI low:', 
      round((ci_low)*100), '%')
print('CI high:', 
      round((ci_high)*100), '%')

print('Standard deviation:', 
      round(np.std(lst_accu_stratified),2))

print('\nNull acccuracy:', 
      round(max(y.mean(), 1 - y.mean())*100), '%')

print('\nClassification error:', 
      round((1 - metrics.accuracy_score(y, y_pred))*100), '%')

print('\nModel sensitivity:', 
      round((TP / float(FN + TP))*100), '%')

print('Model specificity:', 
      round((TN / (TN + FP))*100), '%')

print('\nModel precision:', 
      round((TP / float(TP + FP))*100),'%')

print('Model false positive rate:', 
      round((FP / float(TN + FP))*100),'%')

## ROC curve and AUC value

In [ ]:
# Plot ROC curve

plt.figure(1)

fpr, tpr, n_thresh = metrics.roc_curve(y, scores, pos_label = 1)

plt.plot(fpr,tpr)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')

In [ ]:
# Calculate the AUC value

print(metrics.roc_auc_score(y, scores))

In [ ]:
# Show AUC values for each of the 10 folds

auc_store

In [ ]:
# Save values for figure

with open('scores_xgboost_own_github.npy', 'wb') as f:
    np.save(f, fpr)
    np.save(f, tpr)

# Step 3: Evaluate performance at different sample sizes

In [ ]:
# Define X (predictors) and y (outcome)

X2 = df_sf[df_sf.columns[~df_sf.columns.str.contains('one_year_sf')]]
y2 = df_sf.one_year_sf

# Convert the columns to float 
X2 = X2.astype(np.float)
y2 = y2.astype(np.float)

# Convert to array
X2 = np.array(X2)
y2 = np.array(y2)

random_n = random.randint(0,20)
iterations = np.arange(0,10)
accuracy_batches_iterations=[]
number_iterations = []
for iteration in iterations:
   
    accuracy_batches=[]
    number = []
    index = []
    n_subs = np.linspace(20,700,38).astype(int)

    for ni, nsubs in enumerate(n_subs):
        index = np.random.choice(np.arange(len(X2)),nsubs, replace = False)
        print('There is {} patients'.format(len(index)))
        X2sub = np.array(X2)[index]
        y2sub = np.array(y2)[index]

        # number per batches 
        n_n_sf = len(np.where(y2sub==0)[0])
        print('number not seizure-free: ' + str(n_n_sf))
        n_sf = len(np.where(y2sub==1)[0])
        print('number seizure-free: ' + str(n_sf))

        # Classifier
        model2 = XGBClassifier(base_score = 0.5, booster = 'gbtree', gamma = 0,
                    max_depth = 1, eval_metric = 'error', subsample=0.9,
                    objective = 'binary:logistic')

        # Stratified k fold cross-validation
        scores=np.zeros(len(X2sub))
        y2_pred=np.zeros(len(y2sub))
        y2_pred_fold=[]
        lst_accu_stratified = []
        cm_holder=[]
        for train_index, test_index in skf.split(X2sub, y2sub):
            # split data in stratified fold
            x2_train_fold, x2_test_fold = X2sub[train_index], X2sub[test_index]
            y2_train_fold, y2_test_fold = y2sub[train_index], y2sub[test_index]
            # train on the fold
            model2.fit(x2_train_fold, y2_train_fold)
            scores[test_index] = model2.predict_proba(x2_test_fold)[:,1]
            y2_pred[test_index] = model2.predict(x2_test_fold)
            lst_accu_stratified.append(model2.score(x2_test_fold, y2_test_fold))

        accuracy=((y2_pred==y2sub).sum())/len(y2sub)
        accuracy_batches.append(accuracy)
        number.append(len(y2sub))
    accuracy_batches_iterations.append(accuracy_batches)
    number_iterations.append(number)

In [ ]:
# For iteration in iterations: 

accuracy_batches_mean = np.array(accuracy_batches_iterations).mean(axis=0)
accuracy_batches_std = np.array(accuracy_batches_iterations).std(axis=0)
plt.plot(number_iterations[0], accuracy_batches_mean , color='k')

In [ ]:
# For iteration in iterations: 

accuracy_batches_mean = np.array(accuracy_batches_iterations).mean(axis=0)

x = number_iterations[0]

y_mins=[]
y_maxs=[]

for i in range(0,len(x)):
    y_min, y_max = binom.interval(alpha=0.95, n=x[i], p=accuracy_batches_mean[i])
    y_mins.append(y_min/x[i])
    y_maxs.append(y_max/x[i])

plt.plot(number_iterations[0], accuracy_batches_mean , color='k')
plt.plot(number_iterations[0], y_mins , color='gray')
plt.plot(number_iterations[0], y_maxs , color='gray')

In [ ]:
# Import additional packages

from scipy.stats import powerlaw
from scipy.optimize import curve_fit
from scipy.integrate import quad

In [ ]:
# Implement power law function

y = accuracy_batches_mean

def func_inverse_powerlaw(x, a, b, c):
    return (1-a)-b*x**c

target_func = func_inverse_powerlaw

popt, pcov = curve_fit(target_func, x, y,  maxfev=2000, bounds=[[0,-np.inf,-1],[1,np.inf,0]])

plt.figure(figsize=(10, 7))
plt.plot(x, target_func(x, *popt), '--', color='blue', label='Fitted inverse power power-function ')
plt.plot(x, y, 'ro', label='Learning curve points')
plt.fill_between(x,y_min, y_max,color = 'orange', alpha = 0.15)
plt.legend()
plt.xticks(size = 15)
plt.yticks(size = 15)
plt.ylabel('Accuracy', fontsize = 20)
plt.xlabel('Sample size', fontsize = 20)
plt.legend(loc = 4,prop = {'size': 15})

In [ ]:
# Define functions

x_expand = np.arange(1,2000)
y_est=target_func(x_expand, *popt)
y_mins= np.zeros(len(x_expand))
y_maxs=np.zeros(len(x_expand))
for i,x_exp in enumerate(x_expand):
    y_min, y_max = binom.interval(alpha=0.95, n=x_exp, p=y_est[i])
    y_mins[i] = y_min/x_exp
    y_maxs[i] = y_max/x_exp
y_maxs*=100
y_mins*=100

In [ ]:
# Evaluate performance at larger sample sizes

# Expand x array
y = accuracy_batches_mean

# Create boundaries
popt_min, pcov_min = curve_fit(target_func, x, y_min,  maxfev=2000, bounds=[[0,-np.inf,-1],[1,np.inf,0]])
popt_max, pcov_max = curve_fit(target_func, x, y_max,  maxfev=2000, bounds=[[0,-np.inf,-1],[1,np.inf,0]])
bound_upper = target_func(x_expand, *popt_max)*100
bound_lower = target_func(x_expand, *popt_min)*100

# Plot
plt.figure(figsize=(10, 7))
plt.plot(x, y*100, 'ro', color='blue', label='Learning curve on dataset points')
plt.plot(x, target_func(x, *popt)*100, '-', color='blue', label='Fitted inverse power-function')
plt.plot(x_expand, target_func(x_expand, *popt)*100, '--', color='orange', label='Prediction on expanded dataset')
plt.fill_between(x_expand, y_mins, y_maxs,color = 'orange', alpha = 0.15)
plt.ylim([40,100])
plt.xticks(size = 15)
plt.yticks(size = 15)
plt.ylabel('Accuracy (%)', fontsize = 20)
plt.xlabel('Sample size', fontsize = 20)
plt.legend(loc = 4,prop = {'size': 15})

In [ ]:
# Save values for figure

with open('larger_sample_xgboost_own_github.npy', 'wb') as f:
    np.save(f, x)
    np.save(f, y)
    np.save(f, popt)
    np.save(f, y_mins)
    np.save(f, y_maxs)